In [1]:
import numpy as np
import pandas as pd
import gc #Recolector de basura
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import sklearn.utils
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from sklearn import datasets, linear_model
from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier

In [2]:
test_final_100k_ = pd.read_csv('Datos/test_final_100k.csv')

# Pre procesamiento de los datos


En este primer paso recolectaremos los datos que nos sean útiles para entrenar nuestros modelos y los llevaremos a un formato en el que puedan usarlos.
También realizaremos una limpieza de los datos, eliminando o corrigiendo datos erróneos según corresponda.

## Postulantes

In [24]:
postulantes_educacion_desde_15_Abril = pd.read_csv('Datos/Fiuba_desde_15_Abril/fiuba_1_postulantes_educacion.csv')
postulantes_genero_y_edad_desde_15_Abril = pd.read_csv('Datos/Fiuba_desde_15_Abril/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_educacion_hasta_15_abril = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_1_postulantes_educacion.csv')
postulantes_genero_y_edad_hasta_15_abril = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_educacion_original = pd.read_csv('Datos/datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')
postulantes_genero_y_edad_original = pd.read_csv('Datos/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')

In [25]:
#Concateno datos de los tres sets y filtro postulantes repetidos

postulantes_genero_y_edad = pd.concat([postulantes_genero_y_edad_desde_15_Abril,postulantes_genero_y_edad_hasta_15_abril,postulantes_genero_y_edad_original], axis=0)
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates('idpostulante')

postulantes_educacion = pd.concat([postulantes_educacion_desde_15_Abril,postulantes_educacion_hasta_15_abril,postulantes_educacion_original], axis=0)
postulantes_educacion = postulantes_educacion.drop_duplicates('idpostulante')

In [26]:
#Elimino los dataframes que ya no uso

del [[postulantes_educacion_desde_15_Abril,postulantes_genero_y_edad_desde_15_Abril,postulantes_educacion_hasta_15_abril,postulantes_genero_y_edad_hasta_15_abril,postulantes_educacion_original,postulantes_genero_y_edad_original]]
gc.collect()

118

In [27]:
#Renombro las columnas a un nombre más apropiado

postulantes_educacion = postulantes_educacion.rename(columns={"nombre":"educacion"})

In [28]:
#Hay algun dato nulo?

postulantes_educacion.isnull().any()

idpostulante    False
educacion       False
estado          False
dtype: bool

In [29]:
#Tipos de estado para la educación

postulantes_educacion.estado.value_counts()

Graduado      257355
En Curso      169975
Abandonado     20579
Name: estado, dtype: int64

In [30]:
#Tipos de educación

postulantes_educacion.educacion.value_counts()

Universitario        189379
Secundario           164075
Terciario/Técnico     66251
Otro                  14521
Posgrado               8026
Master                 5319
Doctorado               338
Name: educacion, dtype: int64

In [31]:
#Para cada postulante, me interesa quedarme solamente con la mayor de sus educaciones
#Codifico variables categóricas en forma numérica para que luego puedan procesarlas los algoritmos.

postulantes_educacion['Cod_edu'] = np.array([1 if educacion == 'Secundario' else 
                                             2 if educacion == 'Otro' else 
                                             3 if educacion == 'Terciario/Técnico' else 
                                             4 if educacion == 'Universitario' else 
                                             5 if educacion == 'Posgrado' else 
                                             6 if educacion == 'Master' else 7 for educacion in postulantes_educacion.educacion])

#Ya no necesito la columna educacion
postulantes_educacion = postulantes_educacion.drop('educacion',axis=1)

#Codifico el estado de la educación
postulantes_educacion['Cod_est'] = np.array([1 if estado == 'Abandonado' else 
                                             2 if estado == 'En Curso' else 
                                             3 for estado in postulantes_educacion.estado])
                                   
#Ya no necesito la columna estado
postulantes_educacion = postulantes_educacion.drop('estado',axis=1)


postulantes_educacion = postulantes_educacion.sort_values(['idpostulante','Cod_edu'], ascending = False).drop_duplicates('idpostulante')

In [32]:
postulantes_educacion.head()

,idpostulante,Cod_edu,Cod_est
53092,zvxzzL6,1,3
118722,zvxzvD5,3,2
60153,zvxzrWm,2,3
50863,zvxzmED,1,3
211836,zvxzjoK,3,2


In [33]:
#Sexos declarados

postulantes_genero_y_edad.sexo.value_counts()

FEM           251205
MASC          227853
NO_DECLARA     25342
0.0                7
Name: sexo, dtype: int64

In [34]:
#Codificación de los sexos

postulantes_genero_y_edad['Cod_sexo'] = np.array([0 if sexo == 'MASC' else 
                                                  1 if sexo == 'FEM' else 2 for sexo in postulantes_genero_y_edad.sexo])

#Ya no necesito la columna sexo
postulantes_genero_y_edad = postulantes_genero_y_edad.drop('sexo',axis=1)


In [35]:
#Ojo, fechas de nacimiento nulas

postulantes_genero_y_edad.isnull().any()

idpostulante       False
fechanacimiento     True
Cod_sexo           False
dtype: bool

In [36]:
#Convierto la columna fechanacimiento a formato datetime. 

postulantes_genero_y_edad['fechanacimiento'] = pd.to_datetime(postulantes_genero_y_edad['fechanacimiento'], errors='coerce')

In [37]:
#Fechas nulas

postulantes_genero_y_edad.loc[postulantes_genero_y_edad.fechanacimiento.isnull()].head()

,idpostulante,fechanacimiento,Cod_sexo
1,ZX1,NaT,2
4,N1x,NaT,2
5,8mW,NaT,2
6,52aw,NaT,2
8,a6OE,NaT,2


In [38]:
#Vamos ahora a agregar una columna adicional al dataframe, de modo que podamos observar la edad de cada persona

postulantes_genero_y_edad['edad'] = ((pd.datetime.today() - postulantes_genero_y_edad['fechanacimiento']).dt.days/365)
postulantes_genero_y_edad['edad'] = postulantes_genero_y_edad['edad'].fillna(0)
postulantes_genero_y_edad['edad'] = postulantes_genero_y_edad['edad'].astype(int)
postulantes_genero_y_edad = postulantes_genero_y_edad.drop('fechanacimiento',axis=1)
postulantes_genero_y_edad.head(10)

,idpostulante,Cod_sexo,edad
0,6MM,0,33
1,ZX1,2,0
2,YZP,0,57
3,ebE,0,66
4,N1x,2,0
5,8mW,2,0
6,52aw,2,0
7,NAjM,1,56
8,a6OE,2,0
9,ZjlZ,0,48


In [39]:
#Muchos postulantes con edad cero producto de que no tenian cargada una fecha de nacimiento.

postulantes_genero_y_edad.edad.value_counts().sort_index().head()

0     26096
7         1
9         1
10        1
11        1
Name: edad, dtype: int64

In [40]:
#Hay muchos registros con edad en cero como para simplemente descartarlos. Propongo asignarle a estos registros 
#una edad al azar basándome en el promedio de edades del set de datos y la desviación estándar de las mismas.

#Edad promedio del set de datos
edad_promedio = postulantes_genero_y_edad.loc[postulantes_genero_y_edad.edad !=0].edad.mean()

#Desviación estándar del set de datos
desviacion = postulantes_genero_y_edad.loc[postulantes_genero_y_edad.edad !=0].edad.std()

In [41]:
#Asigno edades

postulantes_genero_y_edad.edad =  np.array([np.random.randint(edad_promedio-desviacion,edad_promedio+desviacion) if edad == 0 else edad for edad in postulantes_genero_y_edad.edad])

In [42]:
#Todos tienen edades

#OBS: Quisiera filtrar a los postulantes cuya edad no está en el intervalo [18,65]. Sin embargo, no puedo hacerlo
#porque test_final_100k_ tiene varios registros pertenecientes a ellos.
#Para los que tienen edad cero, una opción es asignarles edades al azar según la desviación del promedio, la otra es dejarlos omo están.
#Veremos con cuál opción obtenemos mejores resultados.

postulantes_genero_y_edad.edad.value_counts().sort_index()

7          1
9          1
10         1
11         1
12         2
13         1
14         1
15         1
16         9
17       197
18      6446
19     13332
20     17655
21     22600
22     24711
23     26327
24     26837
25     27306
26     27769
27     27349
28     24714
29     23391
30     21638
31     20499
32     18781
33     16672
34     15667
35     14607
36     13944
37     13186
       ...  
62       452
63       316
64       229
65       171
66       148
67       127
68        84
69        59
70        37
71        22
72        21
73        15
74        12
75        12
76         6
77         3
78         6
79         2
80         2
81         1
83         1
85         1
96         1
99         1
101        1
102        2
103        1
116        1
131        1
243        1
Name: edad, Length: 84, dtype: int64

In [43]:
#Merge de los datos de los postulantes

postulantes = pd.merge(postulantes_genero_y_edad,postulantes_educacion,how='left',on='idpostulante')
postulantes['Cod_est'] = postulantes['Cod_est'].fillna('0')
postulantes['Cod_edu'] = postulantes['Cod_edu'].fillna('0')

#Ya no necesito este dataframe
del [[postulantes_genero_y_edad,postulantes_educacion]]
gc.collect()

postulantes.head()

,idpostulante,Cod_sexo,edad,Cod_edu,Cod_est
0,6MM,0,33,0,0
1,ZX1,2,25,0,0
2,YZP,0,57,6,2
3,ebE,0,66,4,3
4,N1x,2,25,0,0


In [44]:
postulantes.isnull().any()

idpostulante    False
Cod_sexo        False
edad            False
Cod_edu         False
Cod_est         False
dtype: bool

In [45]:
postulantes.set_index('idpostulante').to_csv('Datos/postulantes.csv')

# Avisos

In [47]:
avisos_detalle_desde_15_Abril = pd.read_csv('Datos/Fiuba_desde_15_Abril/fiuba_6_avisos_detalle.csv')
avisos_online_hasta_15_abril = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_5_avisos_online.csv')
avisos_detalle_hasta_15_abril = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv')
avisos_online_original = pd.read_csv('Datos/datos_navent_fiuba/fiuba_5_avisos_online.csv')
avisos_detalle_original = pd.read_csv('Datos/datos_navent_fiuba/fiuba_6_avisos_detalle.csv')
avisos_detalle2 = pd.read_csv('Datos/fiuba_6_avisos_detalle_missing_nivel_laboral.csv')

In [48]:
avisos_detalle = pd.concat([avisos_detalle2,avisos_detalle_desde_15_Abril,avisos_detalle_hasta_15_abril,avisos_detalle_original],axis=0).drop_duplicates('idaviso')
avisos_online = pd.concat([avisos_online_hasta_15_abril,avisos_online_original],axis=0).drop_duplicates('idaviso')

In [49]:
#Ya no necesito este dataframe
del [[avisos_detalle_desde_15_Abril,avisos_online_hasta_15_abril,avisos_detalle_hasta_15_abril,avisos_online_original,avisos_detalle_original,avisos_detalle2]]
gc.collect()

65

## Avisos detalle

In [50]:
avisos_detalle.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1001284385,1,Dejanos tu CV,<p><br />Si est&aacute;s interesado en ser par...,Corrientes,NaN,NaN,Full-time,NaN,Otros,Establecimiento Las Marías
1,1110092955,1,para línea de producción (ref. 28376),<p>Empresas vinculadas a la industria automotr...,GBA Oeste,NaN,NaN,Full-time,NaN,Producción,RANDSTAD
2,1110092958,1,para mantenimiento (ref. 27181),"<p style=""display: inline !important;"">Empresa...",GBA Oeste,NaN,NaN,Full-time,NaN,Mantenimiento,RANDSTAD
3,1110391074,1,Trabajá con nosotros,<p><strong>Trabaj&aacute; con nosotros</strong...,Cordoba,NaN,NaN,Full-time,NaN,Ingeniería Industrial,Coca Cola Andina Argentina
4,1110669632,1,Técnico de Mantenimiento,<p>Experiencia en f&aacute;bricas cosm&eacute;...,GBA Oeste,NaN,", Lanús",Full-time,NaN,Mantenimiento,PLUMARI S A


In [51]:
#Hay registros nulos
avisos_detalle.isnull().any()

idaviso                 False
idpais                  False
titulo                  False
descripcion             False
nombre_zona             False
ciudad                   True
mapacalle                True
tipo_de_trabajo         False
nivel_laboral            True
nombre_area             False
denominacion_empresa     True
dtype: bool

In [52]:
#Práctimente ningún registro tiene mapacalle o ciudad, no parecen ser features que me vayan a aportar de cara
#a armar el modelo

print (len(avisos_detalle.loc[avisos_detalle.ciudad.isnull()]))
print (len(avisos_detalle.loc[avisos_detalle.mapacalle.isnull()]))
print (len(avisos_detalle.loc[avisos_detalle.nivel_laboral.isnull()]))
print (len(avisos_detalle.loc[avisos_detalle.denominacion_empresa.isnull()]))

25128
23218
335
7


In [53]:
avisos_detalle=avisos_detalle.fillna(0)

In [54]:
avisos_detalle['mapacalle'] = np.array([0 if mapacalle == 0 else 1 for mapacalle in avisos_detalle.mapacalle])
##################################################################################################################
avisos_detalle['ciudad'] = np.array([0 if ciudad == 0 else 1 for ciudad in avisos_detalle.ciudad])
##################################################################################################################
#Agrego una columna al datframe que me muestre el largo del título del aviso
dic={}
for x in set(avisos_detalle.titulo.values):
    dic[x] = len(x)
    
avisos_detalle['longitud_titulo'] = avisos_detalle['titulo'].map(dic).astype(int)
##################################################################################################################
#Agrego una columna al datframe que me muestre el largo de la descripción del aviso
dic={}
for x in set(avisos_detalle.descripcion.values):
    dic[x] = len(x)
    
avisos_detalle['longitud_descripcion'] = avisos_detalle['descripcion'].map(dic).astype(int)
##################################################################################################################
#Codifico nivel laboral
dic={}
cod=1
for x in set(avisos_detalle.nivel_laboral.values):
    dic[x] = cod
    cod+=1
    
avisos_detalle['nivel_laboral'] = avisos_detalle['nivel_laboral'].map(dic).astype(int)
##################################################################################################################
#Codifico denominacion empresa
dic={}
cod=1
for x in set(avisos_detalle.denominacion_empresa.values):
    dic[x] = cod
    cod+=1
    
avisos_detalle['denominacion_empresa'] = avisos_detalle['denominacion_empresa'].map(dic).astype(int)
##################################################################################################################
#Codifico las zonas
avisos_detalle['nombre_zona'] = avisos_detalle['nombre_zona'].map( {'Gran Buenos Aires':1,
                                    'Capital Federal':2,
                                    'Buenos Aires (fuera de GBA)':3,
                                    'GBA Oeste':4,
                                    'La Plata':5,
                                    'Ciudad de Mendoza':6,
                                    'Mendoza':7,
                                    'Cordoba':8,
                                    'Rosario':9,
                                    'Santa Cruz':10,
                                    'Santa Fe':11,
                                    'San Juan':12,
                                    'Corrientes':13,
                                    'Tucuman':14,
                                    'Catamarca':15,
                                    'Neuquen':16} 
                                  ).astype(int)

##################################################################################################################
#Codifico los tipos de trabajo
avisos_detalle['tipo_de_trabajo'] = avisos_detalle['tipo_de_trabajo'].map( {'Full-time':1,
                                        'Part-time':2,
                                        'Teletrabajo':3,
                                        'Por Horas':4,
                                        'Pasantia':5,
                                        'Temporario':6,
                                        'Por Contrato':7,
                                        'Fines de Semana':8,
                                        'Primer empleo':9,
                                        'Voluntario'  :10
                                        }
                                     ).astype(int)
##################################################################################################################
#Codifico las areas de trabajo
dic={}
cod=1
for x in set(avisos_detalle.nombre_area.values):
    dic[x] = cod
    cod+=1
    
avisos_detalle['nombre_area'] = avisos_detalle['nombre_area'].map(dic).astype(int)

In [55]:
avisos_detalle = avisos_detalle[['idaviso','nombre_zona','tipo_de_trabajo','nombre_area','longitud_titulo','longitud_descripcion','nivel_laboral','denominacion_empresa','mapacalle','ciudad']]
avisos_detalle.head()

,idaviso,nombre_zona,tipo_de_trabajo,nombre_area,longitud_titulo,longitud_descripcion,nivel_laboral,denominacion_empresa,mapacalle,ciudad
0,1001284385,13,1,186,13,116,1,3718,0,0
1,1110092955,4,1,112,39,565,1,1859,0,0
2,1110092958,4,1,18,31,675,1,1859,0,0
3,1110391074,8,1,16,21,576,1,146,0,0
4,1110669632,4,1,18,25,470,1,3408,1,0


## Avisos online 

In [56]:
avisos_online['online']=int(1)

In [57]:
avisos_detalle = pd.merge(avisos_detalle,avisos_online,on='idaviso',how='left')
avisos_detalle['online'] = avisos_detalle['online'].fillna(int(0))

In [58]:
avisos_detalle.isnull().any()

idaviso                 False
nombre_zona             False
tipo_de_trabajo         False
nombre_area             False
longitud_titulo         False
longitud_descripcion    False
nivel_laboral           False
denominacion_empresa    False
mapacalle               False
ciudad                  False
online                  False
dtype: bool

In [59]:
del[avisos_online]
gc.collect()

30

In [60]:
avisos_detalle.head()

,idaviso,nombre_zona,tipo_de_trabajo,nombre_area,longitud_titulo,longitud_descripcion,nivel_laboral,denominacion_empresa,mapacalle,ciudad,online
0,1001284385,13,1,186,13,116,1,3718,0,0,1.0
1,1110092955,4,1,112,39,565,1,1859,0,0,1.0
2,1110092958,4,1,18,31,675,1,1859,0,0,1.0
3,1110391074,8,1,16,21,576,1,146,0,0,1.0
4,1110669632,4,1,18,25,470,1,3408,1,0,1.0


In [61]:
avisos_detalle.set_index('idaviso').to_csv('Datos/avisos_detalle.csv')

## Postulaciones

In [62]:
postulaciones_hasta_15_abril = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_4_postulaciones.csv')
postulaciones_original = pd.read_csv('Datos/datos_navent_fiuba/fiuba_4_postulaciones.csv')

In [63]:
#Concateno datos de todos los sets y filtro repetidos

postulaciones = pd.concat([postulaciones_hasta_15_abril,postulaciones_original],axis=0).drop_duplicates()

In [64]:
#Ya no necesito este dataframe
del [[postulaciones_hasta_15_abril,postulaciones_original]]
gc.collect()

323

In [65]:
postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112248724,NjlD,2018-01-19 07:39:16
1,1112286523,ZaO5,2018-01-24 15:07:39
2,1112272060,ZaO5,2018-01-24 15:20:10
3,1112288401,ZaO5,2018-01-26 08:37:04
4,1112300563,ZaO5,2018-01-30 13:35:48


In [66]:
#No hay registros nulos

postulaciones.isnull().any()

idaviso             False
idpostulante        False
fechapostulacion    False
dtype: bool

In [67]:
postulaciones = pd.merge(postulaciones,avisos_detalle,on='idaviso',how='inner')
postulaciones.head()

,idaviso,idpostulante,fechapostulacion,nombre_zona,tipo_de_trabajo,nombre_area,longitud_titulo,longitud_descripcion,nivel_laboral,denominacion_empresa,mapacalle,ciudad,online
0,1112248724,NjlD,2018-01-19 07:39:16,1,1,153,35,1173,2,2477,0,0,0.0
1,1112248724,MV6eM8N,2018-01-30 23:44:30,1,1,153,35,1173,2,2477,0,0,0.0
2,1112248724,5LzQkN,2018-01-15 07:23:49,1,1,153,35,1173,2,2477,0,0,0.0
3,1112248724,8M2rOBz,2018-01-30 22:20:45,1,1,153,35,1173,2,2477,0,0,0.0
4,1112248724,Dr6V3Da,2018-01-23 15:11:35,1,1,153,35,1173,2,2477,0,0,0.0


In [68]:
#Saco las postulaciones que se repiten
postulaciones = postulaciones.drop_duplicates(subset=['idaviso','idpostulante'])

In [69]:
#Todas las postulaciones historicas

postulaciones_historico = pd.merge(postulantes,postulaciones.drop(['fechapostulacion'],axis=1),on='idpostulante',how='inner')

In [70]:
#Agrego label para indicar si se postulo o no, en este caso todos se postularon

postulaciones_historico['sepostulo'] = 1
postulaciones_historico.head()

,idpostulante,Cod_sexo,edad,Cod_edu,Cod_est,idaviso,nombre_zona,tipo_de_trabajo,nombre_area,longitud_titulo,longitud_descripcion,nivel_laboral,denominacion_empresa,mapacalle,ciudad,online,sepostulo
0,aEW8J,1,53,4,2,1112028437,1,1,157,33,1420,2,4016,0,0,1.0,1
1,aEW8J,1,53,4,2,1112263553,1,1,157,35,1602,3,3084,0,0,0.0,1
2,aEW8J,1,53,4,2,1112262301,1,1,75,53,1537,2,1488,0,0,0.0,1
3,aEW8J,1,53,4,2,1112280883,1,1,75,42,1625,3,3819,0,0,1.0,1
4,aEW8J,1,53,4,2,1112284858,1,1,157,70,2588,2,2256,0,0,0.0,1


In [71]:
#Filtro edades que no sean lógicas

postulaciones_historico = postulaciones_historico.drop(postulaciones_historico.loc[postulaciones_historico.edad < 17].index,axis=0)
postulaciones_historico = postulaciones_historico.drop(postulaciones_historico.loc[postulaciones_historico.edad > 75].index,axis=0)

In [72]:
postulaciones_historico.head()

,idpostulante,Cod_sexo,edad,Cod_edu,Cod_est,idaviso,nombre_zona,tipo_de_trabajo,nombre_area,longitud_titulo,longitud_descripcion,nivel_laboral,denominacion_empresa,mapacalle,ciudad,online,sepostulo
0,aEW8J,1,53,4,2,1112028437,1,1,157,33,1420,2,4016,0,0,1.0,1
1,aEW8J,1,53,4,2,1112263553,1,1,157,35,1602,3,3084,0,0,0.0,1
2,aEW8J,1,53,4,2,1112262301,1,1,75,53,1537,2,1488,0,0,0.0,1
3,aEW8J,1,53,4,2,1112280883,1,1,75,42,1625,3,3819,0,0,1.0,1
4,aEW8J,1,53,4,2,1112284858,1,1,157,70,2588,2,2256,0,0,0.0,1


# Obteniendo los casos de no postulaciones

Ya tengo los casos de postulaciones que se realizaron. Necesito ahora fabricarme casos de postulaciones que no se
hayan realizado para que el modelo no entrene solamente con los casos de exito.
Para que las muestras sean representativas, tomaremos casos de postulantes que hayan visto la publicacion y no see hayan postulado a ella.

## Vistas

In [73]:
vistas_desde_15_Abril = pd.read_csv('Datos/Fiuba_desde_15_Abril/fiuba_3_vistas.csv')
vistas_hasta_15_abril = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_3_vistas.csv')
vistas_original = pd.read_csv('Datos/datos_navent_fiuba/fiuba_3_vistas.csv')

In [74]:
vistas = pd.concat([vistas_desde_15_Abril,vistas_hasta_15_abril,vistas_original],axis=0)
vistas = vistas.drop_duplicates()


del [[vistas_desde_15_Abril, vistas_hasta_15_abril, vistas_original]]
gc.collect()

18

In [75]:
vistas = vistas.rename(columns={'idAviso':'idaviso', 'timestamp':'cantidad_vistas'})
vistas = vistas.groupby(['idaviso','idpostulante']).count().reset_index()
vistas.head()

,idaviso,idpostulante,cantidad_vistas
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [76]:
#Guardo la cantidad de vistas de los postulantes
vistas_postulantes = vistas
vistas_postulantes.set_index('idaviso').to_csv('Datos/vistas_postulantes.csv')
del[[vistas_postulantes]]
gc.collect()

145

In [77]:
#Agrego la cantidad de vistas a las postulaciones
postulaciones_historico = pd.merge(postulaciones_historico,vistas, how='left', on=['idpostulante','idaviso'])
#Aquellos que no figuren con cantidad de vistas se los contara como 1 vista
postulaciones_historico = postulaciones_historico.fillna(1)
postulaciones_historico.head()

,idpostulante,Cod_sexo,edad,Cod_edu,Cod_est,idaviso,nombre_zona,tipo_de_trabajo,nombre_area,longitud_titulo,longitud_descripcion,nivel_laboral,denominacion_empresa,mapacalle,ciudad,online,sepostulo,cantidad_vistas
0,aEW8J,1,53,4,2,1112028437,1,1,157,33,1420,2,4016,0,0,1.0,1,1.0
1,aEW8J,1,53,4,2,1112263553,1,1,157,35,1602,3,3084,0,0,0.0,1,1.0
2,aEW8J,1,53,4,2,1112262301,1,1,75,53,1537,2,1488,0,0,0.0,1,1.0
3,aEW8J,1,53,4,2,1112280883,1,1,75,42,1625,3,3819,0,0,1.0,1,1.0
4,aEW8J,1,53,4,2,1112284858,1,1,157,70,2588,2,2256,0,0,0.0,1,1.0


In [78]:
vistas = pd.merge(vistas, postulaciones_historico, how='left', on=['idpostulante','idaviso','cantidad_vistas'])
vistas.head()

,idaviso,idpostulante,cantidad_vistas,Cod_sexo,edad,Cod_edu,Cod_est,nombre_zona,tipo_de_trabajo,nombre_area,longitud_titulo,longitud_descripcion,nivel_laboral,denominacion_empresa,mapacalle,ciudad,online,sepostulo
0,18,BolNL,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,48375,RwVdKR,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,169730,1KjXB,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,169730,2AKzxa,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,169730,6LJ64,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
vistas = vistas.loc[vistas.edad.isnull()]
vistas.head()

,idaviso,idpostulante,cantidad_vistas,Cod_sexo,edad,Cod_edu,Cod_est,nombre_zona,tipo_de_trabajo,nombre_area,longitud_titulo,longitud_descripcion,nivel_laboral,denominacion_empresa,mapacalle,ciudad,online,sepostulo
0,18,BolNL,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,48375,RwVdKR,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,169730,1KjXB,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,169730,2AKzxa,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,169730,6LJ64,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
#Voy tomando muestras al azar de postulantes y de avisos.
#Con ellas me voy armando combinaciones postulante-aviso

vistas = vistas[['idaviso','idpostulante','cantidad_vistas']].sample(n=len(postulaciones_historico)*2,replace=True,axis=0,random_state=6)

vistas = pd.merge(postulantes,vistas,how='inner',on=['idpostulante'])
vistas = pd.merge(vistas,avisos_detalle,how='inner',on='idaviso') 

vistas['sepostulo'] = 0

#Modifico el orden de las columnas
vistas = vistas[['idpostulante','Cod_sexo','edad','Cod_edu','Cod_est','idaviso','nombre_zona','tipo_de_trabajo','nombre_area','longitud_titulo','longitud_descripcion','nivel_laboral','denominacion_empresa','mapacalle','ciudad','online','sepostulo','cantidad_vistas']]
vistas.head()

,idpostulante,Cod_sexo,edad,Cod_edu,Cod_est,idaviso,nombre_zona,tipo_de_trabajo,nombre_area,longitud_titulo,longitud_descripcion,nivel_laboral,denominacion_empresa,mapacalle,ciudad,online,sepostulo,cantidad_vistas
0,6MM,0,33,0,0,1805750,2,1,153,3,428,5,3404,0,1,0.0,0,2
1,6MM,0,33,0,0,1805750,2,1,153,3,428,5,3404,0,1,0.0,0,2
2,6MM,0,33,0,0,1805750,2,1,153,3,428,5,3404,0,1,0.0,0,2
3,Njx0l,0,48,4,3,1805750,2,1,153,3,428,5,3404,0,1,0.0,0,1
4,Njx0l,0,48,4,3,1805750,2,1,153,3,428,5,3404,0,1,0.0,0,1


In [81]:
vistas.shape

(6806904, 18)

In [82]:
#Junto todo

Data = pd.concat ([postulaciones_historico,vistas],axis=0)
Data.head()

,idpostulante,Cod_sexo,edad,Cod_edu,Cod_est,idaviso,nombre_zona,tipo_de_trabajo,nombre_area,longitud_titulo,longitud_descripcion,nivel_laboral,denominacion_empresa,mapacalle,ciudad,online,sepostulo,cantidad_vistas
0,aEW8J,1,53,4,2,1112028437,1,1,157,33,1420,2,4016,0,0,1.0,1,1.0
1,aEW8J,1,53,4,2,1112263553,1,1,157,35,1602,3,3084,0,0,0.0,1,1.0
2,aEW8J,1,53,4,2,1112262301,1,1,75,53,1537,2,1488,0,0,0.0,1,1.0
3,aEW8J,1,53,4,2,1112280883,1,1,75,42,1625,3,3819,0,0,1.0,1,1.0
4,aEW8J,1,53,4,2,1112284858,1,1,157,70,2588,2,2256,0,0,0.0,1,1.0


In [83]:
#Ya no necesito este dataframe
del [[postulaciones,vistas,postulaciones_historico]]
gc.collect()

474

In [84]:
#Filtro los registros que están presentes en test_final_100k_

Data = Data.set_index(['idpostulante','idaviso']).drop(test_final_100k_.set_index(['idpostulante','idaviso']).index).reset_index()

In [85]:
Data = Data.drop(['idpostulante','idaviso'],axis=1)
Data.head()

,Cod_sexo,edad,Cod_edu,Cod_est,nombre_zona,tipo_de_trabajo,nombre_area,longitud_titulo,longitud_descripcion,nivel_laboral,denominacion_empresa,mapacalle,ciudad,online,sepostulo,cantidad_vistas
0,1,53,4,2,1,1,157,33,1420,2,4016,0,0,1.0,1,1.0
1,1,53,4,2,1,1,157,35,1602,3,3084,0,0,0.0,1,1.0
2,1,53,4,2,1,1,75,53,1537,2,1488,0,0,0.0,1,1.0
3,1,53,4,2,1,1,75,42,1625,3,3819,0,0,1.0,1,1.0
4,1,53,4,2,1,1,157,70,2588,2,2256,0,0,0.0,1,1.0


In [86]:
Data.to_csv('Datos/Data.csv')

# Armado de los datos de entrenamiento y prueba

In [3]:
Data = pd.read_csv('Datos/Data.csv')
Data = Data.drop(['Unnamed: 0','online'], axis=1)

In [4]:
Data.shape

(12950444, 15)

In [6]:
Data.sepostulo.value_counts()

0    2623552
1    2376448
Name: sepostulo, dtype: int64

In [7]:
postulantes = pd.read_csv('Datos/postulantes.csv')
avisos_detalle = pd.read_csv('Datos/avisos_detalle.csv')
vistas_postulantes = pd.read_csv('Datos/vistas_postulantes.csv')

In [8]:
Data.head()

,Cod_sexo,edad,Cod_edu,Cod_est,nombre_zona,tipo_de_trabajo,nombre_area,longitud_titulo,longitud_descripcion,nivel_laboral,denominacion_empresa,mapacalle,ciudad,sepostulo,cantidad_vistas
5834624,1,40,1.0,2.0,1,1,73,53,417,2,2531,0,0,1,1.0
438776,1,29,4.0,3.0,1,1,23,44,1020,6,4027,0,0,1,1.0
8868187,0,25,1.0,3.0,1,2,133,28,521,2,2524,0,0,0,1.0
5366428,1,31,1.0,3.0,1,1,171,71,1664,6,3477,0,0,1,1.0
2856859,1,27,3.0,3.0,1,1,61,26,2276,5,1100,0,0,1,1.0


In [9]:
vistas_postulantes.head()

,idaviso,idpostulante,cantidad_vistas
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [10]:
#Estos son los datos de test_final_100k_ pasados a formato vector. 
#Con estos datos voy a sacar los resultados para llenar el csv del submit.
x_query = pd.merge(test_final_100k_,postulantes,on='idpostulante',how='inner')
x_query = pd.merge(x_query,avisos_detalle,on='idaviso',how='inner')
x_query = pd.merge(x_query,vistas_postulantes,on=['idaviso','idpostulante'],how='left').drop(['idpostulante','idaviso','online'],axis=1)
x_query = x_query.fillna(1)
x_query = x_query.sort_values(by='id')
del[[test_final_100k_]]
gc.collect()
x_query.head()

,id,Cod_sexo,edad,Cod_edu,Cod_est,nombre_zona,tipo_de_trabajo,nombre_area,longitud_titulo,longitud_descripcion,nivel_laboral,denominacion_empresa,mapacalle,ciudad,cantidad_vistas
0,0,1,42,7.0,3.0,2,1,107,19,420,3,3910,0,0,1.0
1,1,0,31,2.0,2.0,2,1,107,19,420,3,3910,0,0,1.0
2,2,1,36,4.0,3.0,2,1,107,19,420,3,3910,0,0,1.0
3,3,0,68,5.0,3.0,2,1,139,43,494,5,3425,0,0,1.0
4,4,1,32,3.0,2.0,2,1,139,43,494,5,3425,0,0,1.0


In [11]:
x_query.shape

(100000, 15)

## Separación en set de train y de test

In [12]:
from sklearn.model_selection import train_test_split

#Acá tengo la totalidad de los datos recolectados y sus labels correspondientes.
x_train_complete = Data.drop(['sepostulo'], axis=1)
y_train_complete = Data['sepostulo']

del [[Data]]
gc.collect()

#Random state es la semilla del random, si la dejamos fija el split que hace de los datos va a ser siempre
#el mismo
x_train, x_test, y_train, y_test = train_test_split(x_train_complete, y_train_complete, test_size = 0.1, random_state = 7)

del [[x_train_complete,y_train_complete]]
gc.collect()

print ("Antes del filtrado")
print (x_train.shape)
print (x_test.shape)

################################################################################################################
#Para no contaminar el resultado al evaluar predicciones, tengo que sacar de x_test los registros que sean identicos a 
#los de x_test.

train=pd.concat([x_train,y_train],axis=1)
test=pd.concat([x_test,y_test],axis=1)
train['tipo'] = 1
test['tipo'] = 0
todo = pd.concat([train,test],axis=0)
todo = todo.sort_values('tipo',ascending=False)
todo = todo.drop_duplicates(todo.columns.drop(['tipo','sepostulo'])) #Con esto desparezco a los tipo 0 que aparecen tambien en train
test = todo.loc[todo.tipo==0]
x_test = test.drop(['sepostulo','tipo'],axis=1)
y_test = test['sepostulo']
x_train = train.drop(['sepostulo','tipo'],axis=1)
y_train = train['sepostulo']

del[[train,test,todo]]
gc.collect()

print ("Despues del filtrado")
print (x_train.shape)
print (x_test.shape)
################################################################################################################


del [[postulantes]]
gc.collect()

Antes del filtrado
(4500000, 14)
(500000, 14)
Despues del filtrado
(4500000, 14)
(150457, 14)


7

In [13]:
#Podemos ver que no hay registros en comun entre x_train y x_test

print (x_train.drop_duplicates().shape)
print (x_test.shape)

print (pd.concat([x_train.drop_duplicates(),x_test],axis=0).drop_duplicates().shape)

(2205450, 14)
(150457, 14)
(2355907, 14)


In [14]:
x_train.shape

(4500000, 14)

In [15]:
x_query.shape

(100000, 15)

In [16]:
x_query.head()

,id,Cod_sexo,edad,Cod_edu,Cod_est,nombre_zona,tipo_de_trabajo,nombre_area,longitud_titulo,longitud_descripcion,nivel_laboral,denominacion_empresa,mapacalle,ciudad,cantidad_vistas
0,0,1,42,7.0,3.0,2,1,107,19,420,3,3910,0,0,1.0
1,1,0,31,2.0,2.0,2,1,107,19,420,3,3910,0,0,1.0
2,2,1,36,4.0,3.0,2,1,107,19,420,3,3910,0,0,1.0
3,3,0,68,5.0,3.0,2,1,139,43,494,5,3425,0,0,1.0
4,4,1,32,3.0,2.0,2,1,139,43,494,5,3425,0,0,1.0


In [17]:
x_train.head()

,Cod_sexo,edad,Cod_edu,Cod_est,nombre_zona,tipo_de_trabajo,nombre_area,longitud_titulo,longitud_descripcion,nivel_laboral,denominacion_empresa,mapacalle,ciudad,cantidad_vistas
1008181,0,45,4.0,3.0,1,1,125,23,1069,2,1488,0,0,1.0
8228665,0,30,4.0,2.0,1,1,176,37,758,2,332,0,0,1.0
11771273,1,29,4.0,2.0,2,1,107,36,1083,2,3162,1,0,1.0
7872646,1,22,4.0,2.0,1,1,73,41,360,2,2456,0,0,1.0
3313814,1,28,4.0,2.0,1,1,176,38,1069,2,159,0,0,1.0


In [18]:
#x_train.to_csv('Datos/x_train.csv')
#pd.DataFrame(y_train).to_csv('Datos/y_train.csv')
#x_test.to_csv('Datos/x_test.csv')
#pd.DataFrame(y_test).to_csv('Datos/y_test.csv')
#x_query.to_csv('Datos/x_query.csv')

# Script para generar submits

In [19]:
def generar_submit(modelo,x_query,nombrecsv,tipo_pred):
    submit=pd.DataFrame()
    submit['id']=x_query['id']
    if tipo_pred == 'proba':
        submit['sepostulo'] = pd.DataFrame(modelo.predict_proba(x_query.drop('id', axis=1)),columns=['Proba_NO','Proba_SI'])['Proba_SI']
    if tipo_pred == 'bin':
        submit['sepostulo'] = modelo.predict(x_query.drop('id', axis=1))
    print (modelo.get_params())
    return submit.set_index('id').to_csv('Datos/'+nombrecsv+'.csv')

# Script para evaluar modelos

In [20]:
def evaluar(modelo, x_test, y_test):
    # Hago las predicciones
    pred = modelo.predict(x_test)
    predicciones = [round(value) for value in pred]

    # Evalúo porcentaje de acierto
    accuracy = accuracy_score(y_test, predicciones)
    print("Porcentaje de acierto: %.2f%%" % (accuracy * 100.0))

# Primera aproximación: métodos simples

Comenzaremos probando dos de los métodos más clásicos: árbol de decisión y KNN, como para tener un primer tanteo de la forma en que las predicciones se comportan

## Árbol de decisión

In [23]:
arbol = tree.DecisionTreeClassifier(random_state=1)
arbol.fit(x_train,y_train)
evaluar(arbol,x_train,y_train)
evaluar(arbol,x_test,y_test)

Porcentaje de acierto: 91.49%
Porcentaje de acierto: 71.95%


In [36]:
#Por lo que vemos mapacalle, ciudad, nombre zona no aportan mucho. Al parecer solamente es ruido.

feature_importances = pd.DataFrame(arbol.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances

,importance
edad,0.195066
longitud_descripcion,0.165457
cantidad_vistas,0.149331
denominacion_empresa,0.120240
longitud_titulo,0.104295
nombre_area,0.089642
Cod_edu,0.053601
Cod_est,0.039581
Cod_sexo,0.032691
nivel_laboral,0.030804


In [26]:
#Probemos entrenar al arbol solamente con los cinco features mas importantes. Mejoro la precision

arbol = tree.DecisionTreeClassifier(random_state=1)
arbol.fit(x_train[['edad','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']],y_train)
evaluar(arbol,x_train[['edad','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']],y_train)
evaluar(arbol,x_test[['edad','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']],y_test)

Porcentaje de acierto: 84.99%
Porcentaje de acierto: 73.96%


### Búqueda de hiperparámetros

Previa lectura de la documentación de sklearn, hemos definido los hiperparámetros del árbol que consideramos más importantes. Definimos también un rango donde buscar, así que ahora haremos uso de random search para probar al azar n combinaciones de los mismos, quedándonos con la mejor.
Cada combinación la entrenaremos usando k_fold para disminuir el riesgo de overfitting.

In [24]:
#Defino un rango para cada parámetro que me intersa, randomSearch va a probar con n combinaciones al azar.
max_depth = [None, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
min_samples_leaf = [1,2,3,4,5,10,15,20,25,30,35,40,45,50]
min_samples_split = [2,3,4,5,6,7,8,9,10,50,100,150,200,250,300]
criterion = ['gini']
splitter = ['random','best']

random_grid = {    
    'max_depth': max_depth,
    'min_samples_leaf': min_samples_leaf,
    'criterion': criterion,
    'splitter': splitter,
    'min_samples_split':min_samples_split,
     }

#El modelo para el que quiero probar parámetros
arbol = tree.DecisionTreeClassifier(random_state=7)

#Pruebo 10 combinaciones al azar y hago k_fold  con k=3 para cada una. Total: 60 iteraciones  (3 minutos aprox). 
#n_jobs maneja la cantidad de procesos(cores) a utilizar. Con -1 estoy usando todos los disponibles.
rf_random = RandomizedSearchCV(estimator = arbol, param_distributions = random_grid, n_iter = 40, cv = 3, verbose=5, n_jobs = -1, random_state=42)
rf_random.fit(x_train[['edad','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']],y_train)

Fitting 3 folds for each of 40 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed: 17.6min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=7,
            splitter='best'),
          fit_params=None, iid=True, n_iter=40, n_jobs=-1,
          param_distributions={'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10, 50, 100, 150, 200, 250, 300], 'splitter': ['random', 'best'], 'criterion': ['gini'], 'max_depth': [None, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100], 'min_samples_leaf': [1, 2, 3, 4, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=5)

In [25]:
#De todas las combinaciones de parámetros que randomSearch probó, esta es la mejor.

print(rf_random.best_params_)
print(rf_random.best_score_)

{'min_samples_split': 100, 'splitter': 'best', 'criterion': 'gini', 'max_depth': 70, 'min_samples_leaf': 20}
0.7963368888888889


In [27]:
evaluar(rf_random.best_estimator_,x_train[['edad','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']],y_train)
evaluar(rf_random.best_estimator_,x_test[['edad','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']],y_test)

Porcentaje de acierto: 80.96%
Porcentaje de acierto: 75.96%


# PCA

Probemos ahora reducir la dimensionalidad de nuestro set de datos.

In [12]:
#Intentemos con PCA. No parece que reducir dimensiones vaya a ayudarme mucho.

#Normalizo la matriz
from sklearn.preprocessing import StandardScaler
x = StandardScaler().fit_transform(x_train.values)

#Veamos cuanta informacion se conserva para cada posible reduccion
for n in range (1,12):
    from sklearn.decomposition import PCA
    # en este caso aplicamos PCA indicando la cantidad de componentes 
    # que queremos, 2, para representar en 2 dimensiones.
    pca = PCA(n_components=n)
    principalComponents = pca.fit_transform(x)
    # reconstruimos el data frame para poder representarlo
    principalDf = pd.DataFrame(data = principalComponents)
    print str(n)+" dimensiones, "+str(sum(pca.explained_variance_ratio_))

1 dimensiones, 0.11989415248733683
2 dimensiones, 0.22297409464297352
3 dimensiones, 0.3223339772042729
4 dimensiones, 0.41694000655820007
5 dimensiones, 0.5104160380732308
6 dimensiones, 0.6009443002103942
7 dimensiones, 0.6874239668656928
8 dimensiones, 0.7708213964001055
9 dimensiones, 0.8499623186834568
10 dimensiones, 0.9256347929937477
11 dimensiones, 1.0


## KNN

Comencemos por analizar uno de los parámetros centrales del algoritmo: el valor de K

In [ ]:
for k in range (1,51,4):
    model = KNeighborsClassifier(n_neighbors=k,
                                 n_jobs=-1,
                                 weights='distance',
                                 p=2)
    model.fit(x_train[['edad','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']], y_train)
    #No es estrictamente necesario usar el set de prueba en KNN
    score = model.score(x_test[['edad','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']],y_test) 
    print("k=%d, Precision=%.2f%%" % (k, score * 100))

k=1, Precision=72.89%
k=3, Precision=74.17%
k=5, Precision=74.88%
k=7, Precision=75.31%
k=9, Precision=75.60%
k=11, Precision=75.70%
k=13, Precision=75.81%
k=15, Precision=75.90%
k=17, Precision=75.90%
k=19, Precision=75.87%
k=21, Precision=75.86%
k=23, Precision=75.89%

In [20]:
#Defino un rango para cada parámetro que me interesa, randomSearch va a probar con n combinaciones al azar.
weights = ['uniform', 'distance']
p = [1,2,3]
n_jobs = [-1]
n_neighbors = [18, 19, 20, 21, 22]

param_grid = {
    'n_neighbors':n_neighbors,
    'weights': weights,
    'p': p,
    'n_jobs': n_jobs
    }

#El modelo para el que quiero probar parámetros
KNN = KNeighborsClassifier()

#Pruebo 20 combinaciones al azar y hago k_fold  con k=3 para cada una. Total: 60 iteraciones  (3 minutos aprox). 
#n_jobs maneja la cantidad de procesos(cores) a utilizar. Con -1 estoy usando todos los disponibles.

grid_search = GridSearchCV(estimator = KNN, 
                         param_grid = param_grid, 
                         cv = 3, 
                         n_jobs = -1, 
                         verbose = 5)
grid_search.fit(x_train[['edad','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']], y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 24.4min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 38.5min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
          fit_params=None, iid=True, n_iter=20, n_jobs=-1,
          param_distributions={'n_neighbors': [18, 19, 20, 21, 22], 'n_jobs': [-1], 'weights': ['uniform', 'distance'], 'p': [1, 2, 3]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [23]:
print(rf_random.best_params_)
print(rf_random.best_score_)

{'n_jobs': -1, 'n_neighbors': 22, 'p': 1, 'weights': 'uniform'}

In [37]:
KNN = KNeighborsClassifier(n_neighbors=150,
                           n_jobs=-1,
                           weights='distance',
                           p=2)
KNN.fit(x_train[['edad','longitud_descripcion','cantidad_vistas','longitud_titulo']], y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=150, p=2,
           weights='distance')

In [ ]:
evaluar(KNN,x_train[['edad','longitud_descripcion','cantidad_vistas','longitud_titulo']],y_train)
evaluar(KNN,x_test[['edad','longitud_descripcion','cantidad_vistas','longitud_titulo']],y_test)

In [23]:
generar_submit(KNN,x_query[['id','cantidad_vistas','longitud_descripcion','denominacion_empresa','longitud_titulo','edad']],'KNN','proba')

{'n_neighbors': 150, 'n_jobs': -1, 'algorithm': 'auto', 'metric': 'minkowski', 'metric_params': None, 'p': 2, 'weights': 'distance', 'leaf_size': 30}


# Métodos de ensamble

Habiendo hecho ya algunas pruebas con métodos más simples, probemos ahora algunos algoritmos más potentes para intentar mejorar nuestros resultados.

## Bagging 

In [ ]:
arbol = tree.DecisionTreeClassifier(max_depth=70,
                                    min_samples_leaf=20,
                                    criterion='gini',
                                    splitter='best',
                                    min_samples_split=100)
arbol.fit(x_train[['edad','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']],y_train)
evaluar(arbol,x_train[['edad','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']],y_train)
evaluar(arbol,x_test[['edad','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']],y_test)

In [ ]:
bagging = BaggingClassifier(base_estimator=arbol, n_estimators=30, max_samples=1.0)
bagging.fit(x_train[['edad','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']],y_train)
evaluar(bagging,x_train[['edad','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']],y_train)
evaluar(bagging,x_test[['edad','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']],y_test)

In [ ]:
generar_submit(bagging,x_query[['id','edad','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']],'baggingtree','proba')

## Random Forest 

In [21]:
RF = RandomForestClassifier(n_estimators=22,
                            min_samples_split = 100, 
                            criterion = 'gini', 
                            max_depth = 70, 
                            min_samples_leaf = 20, 
                            verbose = 2,
                            random_state = 7,
                            n_jobs = -1)

RF.fit(x_train, y_train)
evaluar(RF,x_train,y_train)
evaluar(RF,x_test,y_test)

building tree 1 of 22building tree 2 of 22

building tree 3 of 22
building tree 4 of 22
building tree 5 of 22
building tree 6 of 22
building tree 7 of 22
building tree 8 of 22
building tree 9 of 22
building tree 10 of 22
building tree 11 of 22
building tree 12 of 22
building tree 13 of 22
building tree 14 of 22
building tree 15 of 22
building tree 16 of 22
building tree 17 of 22
building tree 18 of 22
building tree 19 of 22
building tree 20 of 22
building tree 21 of 22
building tree 22 of 22


[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:  2.8min finished
[Parallel(n_jobs=4)]: Done  22 out of  22 | elapsed:   35.2s finished


Porcentaje de acierto: 80.13%


[Parallel(n_jobs=4)]: Done  22 out of  22 | elapsed:    1.1s finished


Porcentaje de acierto: 74.56%


In [23]:
feature_importances = pd.DataFrame(RF.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances

,importance
cantidad_vistas,0.302133
longitud_descripcion,0.188363
denominacion_empresa,0.153146
longitud_titulo,0.134808
nombre_area,0.110512
nivel_laboral,0.029786
edad,0.024466
tipo_de_trabajo,0.018198
Cod_edu,0.013447
Cod_sexo,0.008750


### Búsqueda de hiperparámetros 

In [24]:
from sklearn.ensemble import RandomForestClassifier

#Defino un rango para cada parámetro que me interesa, randomSearch va a probar con n combinaciones al azar.
max_features = ['sqrt','log2']
criterion = ['entropy','gini']
warm_start = [True,False]
oob_score = [True, False]
n_jobs = [-1]
bootstrap = [True]
random_state = [7]

param_grid = {    
    'criterion': criterion,
    'warm_start': warm_start,
    'max_features': max_features,
    'n_estimators':n_estimators,
    'n_jobs':n_jobs,
    'oob_score':oob_score,
    'bootstrap':bootstrap,
    'random_state':random_state
    }

#El modelo para el que quiero probar parámetros
RF = RandomForestClassifier()

#Pruebo 20 combinaciones al azar y hago k_fold  con k=3 para cada una. Total: 60 iteraciones  (3 minutos aprox). 
#n_jobs maneja la cantidad de procesos(cores) a utilizar. Con -1 estoy usando todos los disponibles.

grid_search = GridSearchCV(estimator = RF, 
                         param_grid = param_grid, 
                         cv = 2, 
                         n_jobs = -1, 
                         verbose = 5)
grid_search.fit(x_train[['nombre_area','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']], y_train)

Fitting 2 folds for each of 16 candidates, totalling 32 fits


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done  32 out of  32 | elapsed:  7.3min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  32 out of  32 | elapsed:  7.3min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'warm_start': [True, False], 'n_estimators': [3], 'oob_score': [True, False], 'n_jobs': [-1], 'criterion': ['entropy', 'gini'], 'max_features': ['sqrt', 'log2'], 'random_state': [7], 'bootstrap': [True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

In [23]:
RF = RandomForestClassifier(n_estimators=22,
                            min_samples_split = 100, 
                            criterion = 'entropy', 
                            max_depth = 70, 
                            min_samples_leaf = 20, 
                            verbose = 2,
                            random_state = 7,
                            warm_start = True,
                            oob_score = True,
                            n_jobs = -1,
                            bootstrap = True,
                            max_features = 'sqrt'
                            )

RF.fit(x_train[['nombre_area','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']], y_train)
evaluar(RF,x_train[['nombre_area','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']],y_train)
evaluar(RF,x_test[['nombre_area','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']],y_test)

building tree 1 of 22
building tree 2 of 22
building tree 3 of 22
 building tree 4 of 22
building tree 5 of 22
building tree 6 of 22
building tree 7 of 22
building tree 8 of 22
building tree 9 of 22
building tree 10 of 22
building tree 11 of 22
building tree 12 of 22
building tree 13 of 22
building tree 14 of 22
building tree 15 of 22
building tree 16 of 22
building tree 17 of 22
building tree 18 of 22
building tree 19 of 22
building tree 20 of 22
building tree 21 of 22
building tree 22 of 22


[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:  8.2min finished
[Parallel(n_jobs=4)]: Done  22 out of  22 | elapsed:  1.3min finished


Porcentaje de acierto: 80.59%
Porcentaje de acierto: 70.17%


[Parallel(n_jobs=4)]: Done  22 out of  22 | elapsed:    1.3s finished


In [24]:
generar_submit(RF,x_query[['id','nombre_area','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']],'randomForest','proba')

[Parallel(n_jobs=4)]: Done  22 out of  22 | elapsed:    0.2s finished


{'warm_start': True, 'oob_score': True, 'n_jobs': -1, 'min_impurity_decrease': 0.0, 'verbose': 2, 'max_leaf_nodes': None, 'bootstrap': True, 'min_samples_leaf': 20, 'n_estimators': 22, 'min_samples_split': 100, 'min_weight_fraction_leaf': 0.0, 'criterion': 'entropy', 'random_state': 7, 'min_impurity_split': None, 'max_features': 'sqrt', 'max_depth': 70, 'class_weight': None}


## XGBOOST 

In [28]:
xgb_clf = XGBClassifier(n_estimators=22,
                        max_depth = 70, 
                        silent = False,
                        seed = 4,
                        learning_rate = 0.3)
xgb_clf.fit( x_train, y_train)
evaluar(xgb_clf, x_train, y_train)
evaluar(xgb_clf, x_test, y_test)

C:\Users\Mariano\Anaconda2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Porcentaje de acierto: 91.02%
Porcentaje de acierto: 75.82%


C:\Users\Mariano\Anaconda2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [29]:
feature_importances = pd.DataFrame(xgb_clf.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances

,importance
edad,0.405093
longitud_titulo,0.113481
longitud_descripcion,0.095489
nombre_area,0.086798
denominacion_empresa,0.083089
Cod_edu,0.075830
Cod_est,0.051338
Cod_sexo,0.032482
cantidad_vistas,0.019610
nivel_laboral,0.018260


In [33]:
xgb_clf = XGBClassifier(n_estimators=22,
                        max_depth = 70, 
                        silent = False,
                        seed = 4,
                        learning_rate = 0.3)
xgb_clf.fit( x_train[['edad','longitud_descripcion','nombre_area','denominacion_empresa','longitud_titulo']], y_train)
evaluar(xgb_clf, x_train[['edad','longitud_descripcion','nombre_area','denominacion_empresa','longitud_titulo']], y_train)
evaluar(xgb_clf, x_test[['edad','longitud_descripcion','nombre_area','denominacion_empresa','longitud_titulo']], y_test)

C:\Users\Mariano\Anaconda2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Porcentaje de acierto: 80.58%
Porcentaje de acierto: 74.06%


C:\Users\Mariano\Anaconda2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [36]:
generar_submit(xgb_clf,x_query[['id','edad','longitud_descripcion','nombre_area','denominacion_empresa','longitud_titulo']],'xgboost','proba')

{'reg_alpha': 0, 'colsample_bytree': 1, 'silent': False, 'colsample_bylevel': 1, 'scale_pos_weight': 1, 'learning_rate': 0.3, 'missing': None, 'max_delta_step': 0, 'nthread': -1, 'base_score': 0.5, 'n_estimators': 22, 'subsample': 1, 'reg_lambda': 1, 'seed': 4, 'min_child_weight': 1, 'objective': 'binary:logistic', 'max_depth': 70, 'gamma': 0}


## AdaBoost


In [27]:
adaBoost = AdaBoostClassifier(base_estimator=arbol, n_estimators=30, learning_rate=0.8)
adaBoost.fit(x_train[['edad','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']],y_train)
evaluar(adaBoost,x_train[['edad','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']],y_train)
evaluar(adaBoost,x_test[['edad','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']],y_test)

KeyboardInterrupt: 

In [ ]:
feature_importances = pd.DataFrame(AB.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances

In [33]:
generar_submit(AB,x_query,test_final_100k_,'AdaBoost','binaria')

{'n_estimators': 50, 'base_estimator': None, 'random_state': None, 'learning_rate': 1.0, 'algorithm': 'SAMME.R'}


# Linear regression 

Probemos ahora con un método lineal.

In [129]:
clf = linear_model.LinearRegression(n_jobs=-1, normalize=True,fit_intercept=True)
clf.fit(a_train[['cantidad_vistas','longitud_descripcion','denominacion_empresa','longitud_titulo','nombre_area']], a_target)

VotingClassifier(estimators=[('clf1', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=True)), ('clf2', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=True)), ('clf3', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=True)), ('clf4', LinearRegressio...lize=True)), ('clf8', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=True))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [ ]:
evaluar(voting_clf,x_train[['cantidad_vistas','longitud_descripcion','denominacion_empresa','longitud_titulo','nombre_area']],y_train)
evaluar(voting_clf,x_test[['cantidad_vistas','longitud_descripcion','denominacion_empresa','longitud_titulo','nombre_area']],y_test)

In [17]:
param_grid = {
    'fit_intercept':[True,False], 
    'normalize':[True,False], 
    'copy_X':[True,False],
    'n_jobs':[-1]
}

regr = linear_model.LinearRegression()
grid_search = GridSearchCV(estimator = regr, 
                           param_grid = param_grid, 
                           cv = 3, 
                           n_jobs = -1, 
                           verbose = 5)

# Fit the grid search to the data
grid_search.fit(x_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   33.6s
[Parallel(n_jobs=-1)]: Done  22 out of  24 | elapsed:  1.0min remaining:    5.4s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  1.1min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'normalize': [True, False], 'copy_X': [True, False], 'n_jobs': [-1], 'fit_intercept': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

In [20]:
print grid_search.best_params_
print grid_search.best_score_

{'copy_X': True, 'normalize': False, 'n_jobs': -1, 'fit_intercept': True}
0.0005458686660765686


In [21]:
evaluar(grid_search.best_estimator_, x_train, y_train)
evaluar(grid_search.best_estimator_, x_test, y_test)

Porcentaje de acierto: 57.04%
Porcentaje de acierto: 52.93%


In [14]:
generar_submit(regr,x_query,test_final_100k_,'Linear regression','binaria')

{'copy_X': True, 'normalize': False, 'n_jobs': 1, 'fit_intercept': True}


## VOTING

In [ ]:
voting_clf = VotingClassifier(estimators = [('xgb',xgb_clf),('rf', RF), ('knn',KNN)],voting = 'soft',weights=[3,3,1])
voting_clf.fit(x_train[['edad','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']], y_train)
evaluar(voting_clf,x_train[['edad','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']],y_train)
evaluar(voting_clf,x_test[['edad','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']],y_test)

In [29]:
generar_submit(voting_clf,x_query[['id','edad','longitud_descripcion','cantidad_vistas','denominacion_empresa','longitud_titulo']],'Voting','proba')

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    0.6s finished


{'n_jobs': 1, 'rf__min_impurity_decrease': 0.0, 'knn__n_jobs': -1, 'knn__weights': 'distance', 'flatten_transform': None, 'rf__n_jobs': 1, 'rf__warm_start': False, 'estimators': [('rf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=70, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=20, min_samples_split=100,
            min_weight_fraction_leaf=0.0, n_estimators=22, n_jobs=1,
            oob_score=False, random_state=7, verbose=2, warm_start=False)), ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=41, p=2,
           weights='distance'))], 'rf__bootstrap': True, 'knn__metric': 'minkowski', 'knn__n_neighbors': 41, 'knn__algorithm': 'auto', 'rf': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=70, max_features='auto